# 1. Data Inspectation

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

Acknowledgements

The data is originally from the article Hotel Booking Demand Datasets, written by Nuno Antonio, Ana Almeida, and Luis Nunes for Data in Brief, Volume 22, February 2019.https://www.sciencedirect.com/science/article/pii/S2352340918315191

The data was downloaded and cleaned by Thomas Mock and Antoine Bichat for #TidyTuesday during the week of February 11th, 2020.https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-02-11/readme.md

raw data: https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv

Task: cluster cancel and non- cancel  

Reference:
https://www.kaggle.com/jessemostipak/hotel-booking-demand/notebooks


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [5]:
nulls = df.isnull().sum()
nulls[nulls > 0]
#currently only 4 columns has missing values

children         4
country        488
agent        16340
company     112593
dtype: int64

In [6]:
df = df.replace('Undefined', np.NaN)
# there are few coloumns hace Undefined value instead of NaN,
#replace 'Undefined' to NaN in the dataframe

In [7]:
#calculate the total missing values across the whole dataset
df.isnull().sum().sum()/(len(df.index)*31)
# only have 3.5% missing value, so need to randmomly replace with missing value

0.03528717215739147

# 2. Data Preprocessing for missing values

#### (1) Drop unhelpful columns & rows

In [8]:
percentage = df.isnull().sum()/ len(df)
percentage.sort_values(ascending=False).head()

company                 0.943069
agent                   0.136862
meal                    0.009791
country                 0.004087
distribution_channel    0.000042
dtype: float64

In [9]:
#company has 94.4% missing values, not helpful, drop the column
df.drop(['company'], axis=1, inplace=True)

In [10]:
# reservation_status_date contain a lot of variety 
df.drop(['reservation_status_date'], axis=1, inplace=True)  # objects & 926 varieties 
# reservation_status includes 'Canceled' feature 
#By keeping reservation_status in data,
# it is possible to achieve 100% accuracy rate because that feature is direct way to predict cancellations
# so, drop the reservation_status coumns
df.drop(['reservation_status'], axis=1, inplace=True)


In [11]:
df = df.dropna()

#### (2) inspect all the columns unqiue values for replacing the missing values

In [12]:
#transform column to binary value
df['hotel'] = df['hotel'].map({'Resort Hotel':0, 'City Hotel':1}).astype(int)

df['arrival_date_month'] = df['arrival_date_month'].map({'January':1, 'February': 2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7,
                                                            'August':8, 'September':9, 'October':10, 'November':11, 'December':12}).astype(int)

In [13]:
# Since country colomn has high varity data,so need to transfer to numerical data
# transfer to catergorical data first, then transfer to numeric data
df['country'] = df['country'].astype('category')
df['country'] = df['country'].cat.codes

In [14]:
#inspect data again
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102064 entries, 3 to 119389
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           102064 non-null  int32  
 1   is_canceled                     102064 non-null  int64  
 2   lead_time                       102064 non-null  int64  
 3   arrival_date_year               102064 non-null  int64  
 4   arrival_date_month              102064 non-null  int32  
 5   arrival_date_week_number        102064 non-null  int64  
 6   arrival_date_day_of_month       102064 non-null  int64  
 7   stays_in_weekend_nights         102064 non-null  int64  
 8   stays_in_week_nights            102064 non-null  int64  
 9   adults                          102064 non-null  int64  
 10  children                        102064 non-null  float64
 11  babies                          102064 non-null  int64  
 12  meal            

In [15]:
# create new colomns: 'is_family' , 'deposit_given', 'total_nights'
def family_check(df):
    if ((df['adults'] > 0) & (df['children'] > 0)):
        val = 1
    elif ((df['adults'] > 0) & (df['babies'] > 0)):
        val = 1
    else:
        val = 0
    return val

def deposit(df):
    if ((df['deposit_type'] == 'No Deposit') | (df['deposit_type'] == 'Refundable')):
        return 0
    else:
        return 1
    
def previous_cancellations_check(df):
    if df['previous_cancellations'] == 0:
        return 0
    else:
        return 1
    
def previous_bookings_not_canceled_check(df):
    if df['previous_bookings_not_canceled'] == 0:
        return 1
    else:
        return 0    
    
def booking_changed_check(df):
    if df['booking_changes'] == 0:
        return 0
    else:
        return 1    
    
def feature(df):
    # create new column 'is_family' base on 'adults', 'children', 'babies'
    df['is_family'] = df.apply(family_check, axis = 1)
    # create new column 'deposit_given' base on 'deposit_type'
    df['deposit_given'] = df.apply(deposit, axis=1)
    df['previous_cancelled'] = df.apply(previous_cancellations_check, axis=1)
    df['previous_bookings_not_canceled_check'] = df.apply(previous_bookings_not_canceled_check, axis=1)
    df['booking_changed'] = df.apply(booking_changed_check, axis=1)
    # create new column 'total_nights' base on 'stays_in_weekend_nights' and 'stays_in_week_nights'
    df['total_nights'] = df['stays_in_weekend_nights']+ df['stays_in_week_nights']
    df['booking_times'] = df['previous_cancellations'] + df['previous_bookings_not_canceled']
    return df

df = feature(df)

In [16]:
# since we create 'deposit_given' column from 'deposit_type', so can drop 'deposit_type'
df.drop(['deposit_type'], axis=1, inplace=True)
df.drop(['previous_cancellations'], axis=1, inplace=True)
df.drop(['previous_bookings_not_canceled'], axis=1, inplace=True)
df.drop(['booking_changes'], axis=1, inplace=True)

In [17]:
# check the correlation in the dataset with "is_canceled"
corr_matrix = df.corr()
corr_matrix['is_canceled'].sort_values(ascending=False)
# can see 'arrival_date_month','arrival_date_week_number','arrival_date_year',
#          'children','stays_in_week_nights','arrival_date_day_of_month','total_nights'
# have very low correlation(< 0.01%) with 'is_cancelled', whcih is not significant, 
# so it a good idea to drop these columns for further prediction

is_canceled                             1.000000
deposit_given                           0.460740
country                                 0.295191
previous_cancelled                      0.290006
lead_time                               0.278742
hotel                                   0.114751
booking_times                           0.077117
previous_bookings_not_canceled_check    0.074994
days_in_waiting_list                    0.056020
adults                                  0.036479
adr                                     0.023393
arrival_date_month                      0.008866
stays_in_week_nights                    0.006806
arrival_date_week_number                0.006469
children                                0.005147
arrival_date_year                       0.004027
total_nights                           -0.003762
arrival_date_day_of_month              -0.006036
is_family                              -0.012439
stays_in_weekend_nights                -0.022452
is_repeated_guest   

In [18]:
# drop the colomns with low correlation(< 0.05%) 
# df.drop(['adults', 'adr', 'arrival_date_month','arrival_date_week_number','children','stays_in_week_nights',
#          'arrival_date_year','total_nights','arrival_date_day_of_month','is_family','stays_in_weekend_nights',
#          'is_repeated_guest', 'babies', 'previous_bookings_not_canceled'
#         ], axis=1, inplace=True)
df.drop(['adults','adr','arrival_date_month','arrival_date_week_number',
          'stays_in_week_nights','children','arrival_date_year','total_nights',
        'arrival_date_day_of_month','is_family','stays_in_weekend_nights','babies',
        'is_repeated_guest'], axis=1, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102064 entries, 3 to 119389
Data columns (total 19 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   hotel                                 102064 non-null  int32  
 1   is_canceled                           102064 non-null  int64  
 2   lead_time                             102064 non-null  int64  
 3   meal                                  102064 non-null  object 
 4   country                               102064 non-null  int16  
 5   market_segment                        102064 non-null  object 
 6   distribution_channel                  102064 non-null  object 
 7   reserved_room_type                    102064 non-null  object 
 8   assigned_room_type                    102064 non-null  object 
 9   agent                                 102064 non-null  float64
 10  days_in_waiting_list                  102064 non-null  int64  
 11  

In [20]:
df = pd.get_dummies(data = df, columns = ['meal', 'market_segment', 'distribution_channel',
                                            'reserved_room_type', 'assigned_room_type', 'customer_type'])

In [21]:
#inspect data
df.shape
df.describe()

(102064, 50)

,hotel,is_canceled,lead_time,country,agent,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,deposit_given,previous_cancelled,...,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
count,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,...,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000,102064.000000
mean,0.697415,0.391274,111.966384,89.347439,85.420031,2.567732,0.053173,0.608755,0.121590,0.053731,...,0.061442,0.030520,0.021153,0.005888,0.002665,0.002185,0.039661,0.004625,0.758661,0.197053
std,0.459379,0.488038,107.910821,44.172725,109.527488,18.676237,0.225641,0.800297,0.326814,0.225487,...,0.240140,0.172014,0.143896,0.076510,0.051555,0.046692,0.195163,0.067847,0.427897,0.397775
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,26.000000,51.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,1.000000,0.000000,79.000000,80.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,169.000000,134.000000,229.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,629.000000,173.000000,535.000000,391.000000,3.000000,5.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### (3). Data Preprocessing for ML

In [22]:
# feature set and targer set
X = df.drop('is_canceled', axis = 1)
y = df['is_canceled']

In [23]:
# split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

#### (4) Data scaling

##### Choosing scaling
##### use MixMax scaling in order to normalize the data set. The data in our data set are spread across a wide range of values, which might result in various features affecting the final result more than the other feature. MixMax scaling reduces this effect by re-scaling the data to a specificed range of values, in this case 0-1

In [24]:
#data scalering - StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

MinMaxScaler()

# 3. Classification

####  Evaluation strategy: The training set score and test set scores are close, the highest test set score among all models is better

###  (1) KNN Classification

In [25]:
#GridsearchCV and cross validation searching for KNN hypterparameter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 11)}
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, param_grid, cv=5,return_train_score = False,n_jobs = -1)# return_train_score = True,
knn_cv.fit(X_train, y_train)
knn_cv.best_params_

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])})

{'n_neighbors': 10}

In [26]:
import pandas as pd
knn_cv_result = pd.DataFrame(knn_cv.cv_results_)
knn_cv_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,15.317624,1.601019,29.185925,1.077996,1,{'n_neighbors': 1},0.823122,0.820379,0.819138,0.821151,0.826638,0.822086,0.002618,10
1,13.680802,3.146731,33.933625,1.106967,2,{'n_neighbors': 2},0.823841,0.824755,0.822469,0.824744,0.825919,0.824346,0.001147,9
2,9.553842,1.979947,36.945966,1.141774,3,{'n_neighbors': 3},0.827564,0.825669,0.826845,0.829969,0.830949,0.828199,0.001966,8
3,9.195402,2.546591,40.437226,1.786006,4,{'n_neighbors': 4},0.828609,0.830176,0.829915,0.830035,0.832060,0.830159,0.001104,4
4,8.984166,2.086268,44.658135,2.471072,5,{'n_neighbors': 5},0.829262,0.827694,0.825865,0.831406,0.832060,0.829258,0.002299,7


In [27]:
#k=10 is the best hyperparameter, applied this value in the model
clf = KNeighborsClassifier(n_neighbors=10,n_jobs = -1)
clf.fit(X_train, y_train)
print("Training set score: {:.3f}".format(clf.score(X_train, y_train)))
print("Test set accuracy: {:.3f}".format(clf.score(X_test, y_test)))
print("Best parameters: {}".format(knn_cv.best_params_))
print("Best cross-validation score: {:.4f}".format(knn_cv.best_score_))

KNeighborsClassifier(n_jobs=-1, n_neighbors=10)

Training set score: 0.861
Test set accuracy: 0.836
Best parameters: {'n_neighbors': 10}
Best cross-validation score: 0.8316


### (2) Logistic Regression

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":[0.001, 0.01, 0.1, 1, 10, 100, 1000], "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(solver='liblinear')
logreg_cv=GridSearchCV(logreg,grid,cv=5,return_train_score = False ,n_jobs = -1)
logreg_cv.fit(X_train,y_train)
logreg_cv.best_params_

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']})

{'C': 1, 'penalty': 'l2'}

In [29]:
import pandas as pd
logreg_cv_result = pd.DataFrame(logreg_cv.cv_results_)
logreg_cv_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.340091,0.019806,0.019946,0.006735,0.001,l1,"{'C': 0.001, 'penalty': 'l1'}",0.728021,0.729066,0.732528,0.731269,0.726370,0.729451,0.002212,14
1,0.444013,0.021772,0.016756,0.012779,0.001,l2,"{'C': 0.001, 'penalty': 'l2'}",0.748922,0.752776,0.750359,0.751061,0.748122,0.750248,0.001634,13
2,5.392576,0.438944,0.007779,0.000746,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.785434,0.787916,0.780340,0.784571,0.786400,0.784932,0.002551,11
3,0.879050,0.060812,0.007780,0.000399,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.780862,0.781450,0.777335,0.778954,0.780195,0.779759,0.001469,12
4,71.721738,6.670462,0.009774,0.003116,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",0.792619,0.795363,0.787524,0.792998,0.792279,0.792157,0.002556,7


In [32]:
#use best parameter C value 
logreg1 = LogisticRegression(C=1, penalty = 'l2',solver='liblinear',random_state=0).fit(X_train, y_train)
print("Training set score: {:.3f}".format(logreg1.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg1.score(X_test, y_test)))
print("Best parameters: {}".format(logreg_cv.best_params_))
print("Best cross-validation score: {:.4f}".format(logreg_cv.best_score_))

Training set score: 0.793
Test set score: 0.792
Best parameters: {'C': 1, 'penalty': 'l2'}
Best cross-validation score: 0.7924


### (3) Linear Support Vector Machine

In [33]:
#select best hyperparameter #slow
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
Cs = [0.01, 0.1, 1, 10, 100]
param_grid = {'C': Cs}
linearSVC = GridSearchCV(LinearSVC(max_iter=500000), param_grid, cv=5,return_train_score = False,n_jobs = -1)
linearSVC.fit(X_train, y_train)
linearSVC.best_params_

GridSearchCV(cv=5, estimator=LinearSVC(max_iter=500000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100]})

{'C': 10}

In [34]:
import pandas as pd
linearSVC_result = pd.DataFrame(linearSVC.cv_results_)
linearSVC_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.553121,0.025109,0.022739,0.007421,0.01,{'C': 0.01},0.787982,0.790594,0.783932,0.789274,0.788752,0.788107,0.002254,5
1,1.057771,0.032105,0.019548,0.012060,0.1,{'C': 0.1},0.790986,0.793468,0.786545,0.791822,0.791299,0.790824,0.002304,4
2,7.216894,1.033019,0.007380,0.000798,1,{'C': 1},0.791444,0.794513,0.786871,0.791822,0.791561,0.791242,0.002461,3
3,65.159093,7.330288,0.007580,0.000489,10,{'C': 10},0.791509,0.794579,0.786871,0.791822,0.791822,0.791320,0.002488,1
4,472.140257,54.768373,0.004787,0.000977,100,{'C': 100},0.791509,0.794579,0.786806,0.791822,0.791822,0.791307,0.002511,2


In [35]:
#use best parameter C value 
svm = LinearSVC(C=10).fit(X_train, y_train)
print("Training set score: {:.3f}".format(svm.score(X_train, y_train)))
print("Test set score: {:.3f}".format(svm.score(X_test, y_test)))
print("Best parameters: {}".format(linearSVC.best_params_))
print("Best cross-validation score: {:.4f}".format(linearSVC.best_score_))

Training set score: 0.791
Test set score: 0.791
Best parameters: {'C': 10}
Best cross-validation score: 0.7913


C:\Users\sylvi\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### (4) Kerenilzed Support Vector Machine (rbf, poly, and linear)

In [36]:
#linear hyperparameter selection 
from sklearn.svm import SVC
Cs = [0.01, 0.1, 1, 10, 100]
param_grid = {'C': Cs}
kerenl_lin = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5,return_train_score = False,n_jobs = -1)
kerenl_lin.fit(X_train, y_train)
print("The best classifier is: ", kerenl_lin.best_params_)

GridSearchCV(cv=5, estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100]})

The best classifier is:  {'C': 10}


In [37]:
import pandas as pd
kerenl_lin_result = pd.DataFrame(kerenl_lin.cv_results_)
kerenl_lin_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,489.791783,3.158449,65.479635,1.127286,0.01,{'C': 0.01},0.747812,0.751470,0.756368,0.752890,0.747012,0.751110,0.003424,5
1,478.342641,2.759827,61.122690,0.992443,0.1,{'C': 0.1},0.790333,0.792489,0.786610,0.791756,0.791495,0.790537,0.002082,4
2,500.905225,2.288524,59.526759,1.365252,1,{'C': 1},0.792554,0.796408,0.788439,0.794631,0.795153,0.793437,0.002791,3
3,659.033178,4.103275,57.766269,0.971834,10,{'C': 10},0.792685,0.796995,0.788635,0.794761,0.795676,0.793750,0.002918,1
4,1206.418866,157.122317,28.532873,7.328467,100,{'C': 100},0.792489,0.796995,0.788570,0.794892,0.795414,0.793672,0.002933,2


In [49]:
#use best parameter C value 
svc = SVC(kernel='linear', C=10,gamma='auto').fit(X_train, y_train)
print("Training set score: {:.3f}".format(svc.score(X_train, y_train)))
print("Test set score: {:.3f}".format(svc.score(X_test, y_test)))
print("Best parameters: {}".format(kerenl_lin.best_params_))
print("Best cross-validation score: {:.4f}".format(kerenl_lin.best_score_))

Training set score: 0.795
Test set score: 0.795
Best parameters: {'C': 10}
Best cross-validation score: 0.7938


In [39]:
#rbf hypterparameter selection 
C_range = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = {'C': C_range}
kernel_rbf = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5,return_train_score = False, n_jobs = -1)
kernel_rbf.fit(X_train, y_train)
print("The best classifier is: ", kernel_rbf.best_estimator_)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

The best classifier is:  SVC(C=100)


In [40]:
import pandas as pd
kernel_rbf_result = pd.DataFrame(kernel_rbf.cv_results_)
kernel_rbf_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,751.106065,1.947119,120.593491,0.530850,0.001,{'C': 0.001},0.724494,0.726649,0.729980,0.728526,0.724410,0.726812,0.002197,6
1,579.101342,50.476863,85.930069,12.361693,0.01,{'C': 0.01},0.745918,0.747159,0.748988,0.747926,0.744399,0.746878,0.001594,5
2,596.312867,21.544042,84.930833,3.988337,0.1,{'C': 0.1},0.799935,0.801437,0.794448,0.800705,0.799922,0.799289,0.002485,4
3,560.773796,5.220066,79.712361,1.489428,1,{'C': 1},0.820183,0.816395,0.811234,0.816578,0.822523,0.817383,0.003839,2
4,612.733003,15.130885,71.716142,2.383756,10,{'C': 10},0.815807,0.817178,0.810973,0.819257,0.821869,0.817017,0.003649,3


In [50]:
#use best parameter C value 
svc = SVC(kernel='rbf', C=100,gamma='auto').fit(X_train, y_train)
print("Training set score: {:.3f}".format(svc.score(X_train, y_train)))
print("Test set score: {:.3f}".format(svc.score(X_test, y_test)))
print("Best parameters: {}".format(kernel_rbf.best_params_))
print("Best cross-validation score: {:.4f}".format(kernel_rbf.best_score_))

Training set score: 0.818
Test set score: 0.820
Best parameters: {'C': 100}
Best cross-validation score: 0.8195


In [42]:
#poly hyperparameter selection ##slow
from sklearn.svm import SVC
Cs = [0.01, 0.1, 1, 10, 100]
param_grid = {'C': Cs}
kernel_poly = GridSearchCV(SVC(kernel='poly'), param_grid, cv=5,return_train_score = False,n_jobs = -1)
kernel_poly.fit(X_train, y_train)
print("The best classifier is: ", kernel_poly.best_params_)

GridSearchCV(cv=5, estimator=SVC(kernel='poly'), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100]})

The best classifier is:  {'C': 100}


In [43]:
import pandas as pd
kernel_poly_result = pd.DataFrame(kernel_poly.cv_results_)
kernel_poly_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,643.114877,2.792278,88.960723,0.876453,0.01,{'C': 0.01},0.747485,0.749314,0.750555,0.750539,0.746162,0.748811,0.001735,5
1,597.875687,5.725208,82.071811,1.518693,0.1,{'C': 0.1},0.811430,0.806074,0.801698,0.807760,0.811287,0.807650,0.003618,4
2,588.698921,4.461562,73.101244,2.776900,1,{'C': 1},0.819660,0.817897,0.811626,0.818930,0.822719,0.818166,0.003644,2
3,683.837155,4.126800,69.772625,2.872622,10,{'C': 10},0.815741,0.817374,0.811365,0.819322,0.820432,0.816847,0.003179,3
4,1021.243759,116.552396,48.529882,5.499616,100,{'C': 100},0.815676,0.818746,0.812280,0.823503,0.822523,0.818545,0.004193,1


In [44]:
#use best parameter C value 
svc = SVC(kernel = 'poly',C=100,gamma='auto').fit(X_train, y_train)
print("Training set score: {:.3f}".format(svc.score(X_train, y_train)))
print("Test set score: {:.3f}".format(svc.score(X_test, y_test)))
print("Best parameters: {}".format(kernel_poly.best_params_))
print("Best cross-validation score: {:.4f}".format(kernel_poly.best_score_))

Training set score: 0.816
Test set score: 0.818
Best parameters: {'C': 100}
Best cross-validation score: 0.8185


### (5) Decision Tree

In [45]:
#decision tree hyperparameter
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
parameters={'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)}
clf_tree=DecisionTreeClassifier()
grid_search=GridSearchCV(clf_tree,parameters, cv=10,return_train_score = False,n_jobs = -1)
grid_search.fit(X_train, y_train)
print("The best classifier is: ", grid_search.best_params_)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 20, 2),
                         'min_samples_split': range(10, 500, 20)})

The best classifier is:  {'max_depth': 17, 'min_samples_split': 10}


In [46]:
import pandas as pd
grid_search_result = pd.DataFrame(grid_search.cv_results_)
grid_search_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.324332,0.050967,0.007380,0.006927,1,10,"{'max_depth': 1, 'min_samples_split': 10}",0.725016,0.731025,0.728543,0.729589,0.734814,0.730242,0.731809,0.730764,0.730598,0.722106,0.729451,0.00339,226
1,0.336100,0.073086,0.005386,0.001424,1,30,"{'max_depth': 1, 'min_samples_split': 30}",0.725016,0.731025,0.728543,0.729589,0.734814,0.730242,0.731809,0.730764,0.730598,0.722106,0.729451,0.00339,226
2,0.377888,0.096174,0.005586,0.002239,1,50,"{'max_depth': 1, 'min_samples_split': 50}",0.725016,0.731025,0.728543,0.729589,0.734814,0.730242,0.731809,0.730764,0.730598,0.722106,0.729451,0.00339,226
3,0.362828,0.095909,0.004787,0.000746,1,70,"{'max_depth': 1, 'min_samples_split': 70}",0.725016,0.731025,0.728543,0.729589,0.734814,0.730242,0.731809,0.730764,0.730598,0.722106,0.729451,0.00339,226
4,0.316554,0.048829,0.005684,0.002046,1,90,"{'max_depth': 1, 'min_samples_split': 90}",0.725016,0.731025,0.728543,0.729589,0.734814,0.730242,0.731809,0.730764,0.730598,0.722106,0.729451,0.00339,226


In [47]:
#use best parameters values 
clf_tree=DecisionTreeClassifier(max_depth=17,min_samples_split=10).fit(X_train, y_train)
print("Training clf_treeset score: {:.3f}".format(clf_tree.score(X_train, y_train)))
print("Test set score: {:.3f}".format(clf_tree.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Training clf_treeset score: 0.868
Test set score: 0.851
Best parameters: {'max_depth': 17, 'min_samples_split': 10}
Best cross-validation score: 0.8480


### (6) Random Forest

In [53]:
# random forest hyperparameter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameters={'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)}
clf_treeR=RandomForestClassifier()
grid_searchR=GridSearchCV(clf_treeR,parameters, cv=10,return_train_score = False,n_jobs = -1)
grid_searchR.fit(X_train, y_train)
print("The best classifier is: ", grid_searchR.best_params_)

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(1, 20, 2),
                         'min_samples_split': range(10, 500, 20)})

The best classifier is:  {'max_depth': 19, 'min_samples_split': 10}


In [54]:
import pandas as pd
grid_searchR_result = pd.DataFrame(grid_searchR.cv_results_)
grid_searchR_result.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,3.815095,0.217388,0.102526,0.013861,1,10,"{'max_depth': 1, 'min_samples_split': 10}",0.733508,0.742782,0.742391,0.742391,0.742782,0.743305,0.742521,0.741476,0.743271,0.732689,0.740712,0.003842,227
1,3.919515,0.352249,0.098337,0.012648,1,30,"{'max_depth': 1, 'min_samples_split': 30}",0.733769,0.736643,0.681385,0.719399,0.745265,0.739386,0.741084,0.743828,0.746799,0.737523,0.732508,0.018561,243
2,3.622309,0.179460,0.094349,0.007572,1,50,"{'max_depth': 1, 'min_samples_split': 50}",0.740039,0.745787,0.742391,0.682430,0.751666,0.741607,0.738472,0.741215,0.663575,0.735171,0.728235,0.028242,249
3,3.806317,0.231894,0.105119,0.019863,1,70,"{'max_depth': 1, 'min_samples_split': 70}",0.733116,0.740170,0.742391,0.742391,0.743174,0.736643,0.745918,0.743697,0.731252,0.732297,0.739105,0.005067,228
4,3.678958,0.244767,0.112201,0.020615,1,90,"{'max_depth': 1, 'min_samples_split': 90}",0.738994,0.743174,0.736251,0.683867,0.749706,0.739909,0.744219,0.740170,0.742226,0.733473,0.735199,0.017622,236


In [55]:
#use best parameters values 
clf_treeR=RandomForestClassifier(max_depth=19,min_samples_split=10).fit(X_train, y_train)
print("Training clf_treeset score: {:.3f}".format(clf_treeR.score(X_train, y_train)))
print("Test set score: {:.3f}".format(clf_treeR.score(X_test, y_test)))
print("Best parameters: {}".format(grid_searchR.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_searchR.best_score_))

Training clf_treeset score: 0.875
Test set score: 0.864
Best parameters: {'max_depth': 19, 'min_samples_split': 10}
Best cross-validation score: 0.8597


# 4. Find the best model

##### 1. Knn: train 0.861, test 0.836, Best cross-validation score 0.8316


##### 2. Logistic Regression:  train  0.793,  test 0.792, Best cross-validation score 0.7924


##### 3. Linear Support Vector Machine : train 0.791, test 0.791, Best cross-validation score: 0.7913

##### 4.Kerenilzed Support Vector Machine (rbf, poly, and linear):   
##### (1) Linear train 0.795, test 0.795, Best cross-validation score: 0.7938 
##### (2) Rbf train 0.818, test 0.820, Best cross-validation score: 0.8195
##### (3) Poly train 0.816 test 0.818, Best cross-validation score: 0.8185

#### 5. decision tree: train 0.868  test 0.851, Best cross-validation score: 0.8480

#### 6. random forest: train 0.875  test 0.864, Best cross-validation score: 0.8597

### Random forest model has the highest training and test score. In addition, it also has the highest Best cross-validation score-- 0.8597. Therefore, Random forest model is the best option

In [56]:
from sklearn.metrics import accuracy_score, roc_auc_score

y_pred = grid_searchR.predict(X_test)
print('accuracy_score: ', accuracy_score(y_test, y_pred))
print('roc_auc_score: ', roc_auc_score(y_test, grid_searchR.predict_proba(X_test)[:,1]))

accuracy_score:  0.863183884621414
roc_auc_score:  0.9387300058724304
